# This notebook allows to analyze data and get useful metrics from redis and google-analytics/firebase

In [1]:
# Specify known utm values to filter data
known_utm_values = {
    "utm_source": [
        "linkedin",
        "email",
        "telegram",
        "cv" # If someone sends my pdf
    ],
    "utm_campaign": [
        "cold_message", # Just send message to someone I known directly
        "apply_directly", # Apply for job directy through linkedin, remoteok or other platform
        "apply_hr", # Find HR in company that is hiring
        "my_profile_page", # Links that are placed in my profile pages
        "backend-developer-v1", # Used with utm_source=cv
        "product-manager-v1" # Used with utm_source=cv
    ]
}


# Specify known firebase dynamic links
known_dynamic_links = {
    "https://artbred.page.link/XktS": {
        "utm_params": { # Or use bigquery, but it doesn't have clicks and redirects
            "utm_source": "linkedin",
            "utm_campaign": "test"
        }
    }
}

# Make analytics for certain period of time
duration_days = 30

# Set true if there is no data available
mock = True

# Timezone
timezone = "Etc/UTC"

In [12]:
# Create combinations from utm_source, utm_campaign
from itertools import permutations

def permutations_generator(dictionary):
    keys = list(dictionary.keys())
    print(dictionary[keys[0]], dictionary[keys[1]])
    for permutation in permutations(dictionary[keys[0]], len(dictionary[keys[1]])):
        yield permutation

utm_permutations = permutations_generator(known_utm_values)

for combination in utm_permutations:
    print(combination)

['linkedin', 'email', 'telegram', 'cv'] ['cold_message', 'apply_directly', 'apply_hr', 'my_profile_page', 'backend-developer-v1', 'product-manager-v1']


In [3]:
!pip3 install python-dotenv

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [4]:
import os, requests

from dotenv import load_dotenv
from pprint import pprint
from random import randrange
load_dotenv()

SIMPLE_ANALYTICS_API_KEY = os.getenv('SIMPLE_ANALYTICS_API_KEY')
SIMPLE_ANALYTCIS_USER_ID = os.getenv('SIMPLE_ANALYTCIS_USER_ID')


fields_stats_for_simple_analytics = [
    "pageviews", # the total amount of page views in the specified period
    "visitors", # the total amount of visitors (unique page views) in the specified period
    "histogram", # an array with page views and visitors per day
    "countries", # a list of country codes
    "utm_sources",
    "utm_campaigns",
    "seconds_on_page" # the median of seconds a visitor spent on the page
]


def mock_simple_analytics_stats(stats):
    # Reset counters
    stats['pageviews'] = 0
    stats['visitors'] = 0
    
    for histogram in stats['histogram']:
        page_views = randrange(0, 100)
        visitors = page_views // randrange(3, 5)
        
        histogram['pageviews'] = page_views
        histogram['visitors'] = visitors
        
        # Add to overall stats
        stats['pageviews'] += page_views
        stats['visitors'] += visitors
    
    
    stats['seconds_on_page'] = randrange(1, 35)
    
    for utm_campaign in stats['utm_campaigns']:
        utm_campaign[''] = utm_campaign
    
    return stats
    

def get_simple_analytics_stats(start=None, end=None):
    fields_stats_for_simple_analytics_str = ','.join(fields_stats_for_simple_analytics)
    url = f"https://simpleanalytics.com/artbred.io.json?version=5&fields={fields_stats_for_simple_analytics_str}&timezone={timezone}"
    
    # By default returns data for last 30 days
    if start and end:
        url += f"&start={start}&end={end}"

    response = requests.get(url, headers={
        "Content-Type": "application/json",
        "Api-Key": SIMPLE_ANALYTICS_API_KEY,
        "User-Id": SIMPLE_ANALYTCIS_USER_ID
    })
        
    stats = response.json()
    if not stats['ok']:
        raise ValueError(stats)
    
    if mock:
        stats = mock_simple_analytics_stats(stats)
    
    return stats
    
    
simple_analytics_stats = get_simple_analytics_stats()
pprint(simple_analytics_stats)

{'__countries': 'INFO: Top 50 countries',
 '__generated_in_ms': 'INFO: Time needed to generate this response on the '
                      'server in milliseconds',
 '__histogram': 'INFO: This histogram is exactly the same as the chart in your '
                'dashboard',
 '__info': "INFO: Disable these info fields by adding 'info=false' to the URL.",
 '__ok': 'INFO: With this boolean you can quickly check if you can expect a '
         'response',
 '__pageviews': 'INFO: Just a total of all pageviews',
 '__path': 'INFO: You can add a path to your domain (e.g. '
           'example.com/contact.json or via the pages param). counts are '
           'relative to only that path.',
 '__seconds_on_page': 'INFO: The median time on page in seconds '
                      '(https://docs.simpleanalytics.com/explained/time-on-page)',
 '__timezone': 'INFO: You can change your timezone in your website settings',
 '__utm_campaigns': 'INFO: Top 50 UTM campaigns',
 '__utm_sources': 'INFO: Top 50 UTM